# Convert TEI into TSV


In [1]:
import re
import os
from lxml import etree
import pandas as pd

In [18]:
def clean_text(text):
    # Remove leading and trailing whitespace
    text = text.strip()
    # Replace newlines with a space
    text = text.replace("\n", " ")
    # Split the text into words and join them with a single space
    text = " ".join(text.split())
    return text

def extract_text(element):
    text = element.text or ""
    for child in element:
        text += extract_text(child)
        if child.tail:
            text += child.tail
    return clean_text(text)


def clean_extracted_text(raw_text):
    """
    Remplace tous les sauts de ligne et tabulations par un espace simple.
    """
    return " ".join(raw_text.split())


def extract_xml_content(element):
    """
    Parcourt l'arbre XML. 
    Retourne une chaîne brute (avec \n et espaces originaux).
    Gère la transformation des balises enfants <hi rend="italic">.
    """
    # 1. On commence avec le texte direct de l'élément (s'il existe)
    content = [element.text or ""]
    
    for child in element:
        # 2. Appel récursif : on récupère le contenu "transformé" de l'enfant
        child_content = extract_xml_content(child)
        
        # 3. C'est ici que le PARENT décide d'habiller l'enfant
        # On utilise endswith pour ignorer les namespaces (ex: {http://tei...}hi)
        if child.tag.endswith('hi') and child.get('rend') == 'italic':
            child_content = f"<i>{child_content}</i>"
        elif child.tag.endswith('pb'): # equal do not work because of namespaces
            #print("Found <pb> tag with attributes:", child.attrib)
            n = child.get('n')
            # if n not pair
            if n and int(n) % 2 != 0:
                child_content = f'<pb n="{n}"/>'
        
        content.append(child_content)
        
        # 4. On ajoute le "tail" (le texte qui suit la balise fermante de l'enfant)
        if child.tail:
            content.append(child.tail)
            
    # On joint tout sans toucher aux espaces pour l'instant
    return clean_extracted_text("".join(content))


In [ ]:
### !!! CODE BELOW IS FOR PARAGRAPH LEVEL (NOT SUBENTRY LEVEL)

inputpath = os.path.join('..', 'data', '1743_LeRobert', 'tei')

data = []

for filename in sorted(os.listdir(inputpath)):
    try:
        parser = etree.XMLParser(collect_ids=False, encoding='utf-8') 
        root = etree.parse(os.path.join(inputpath, filename), parser=parser).getroot()    
        #print(root.nsmap)
        print(filename)
        volume = filename[2:3]

        for entry in root.findall('.//entry[@type="mainEntry"]', namespaces=root.nsmap):
            
            id = 1
            entry_id = entry.get('id')
            form = entry.find('.//form[@type="lemma"]/orth', namespaces=root.nsmap)
            if form is not None:
                entry_lemma = form.text
            else:
                print("Forme : non trouvée, entry:", entry_id)
                entry_lemma = None
            
            for i, paragraph in enumerate(entry.findall('./p', namespaces=root.nsmap)):
                usg = None
                content = ""
                html = ""
                form = paragraph.find('.//form/orth', namespaces=root.nsmap)
                if form is not None:
                    paragraph_lemma = form.text
                else:
                    paragraph_lemma = None
                
                # get number of usg with type="domain" in the paragraph
                num_usg_domain = len(paragraph.findall('.//usg[@type="domain"]', namespaces=root.nsmap))

                usg = paragraph.find('.//usg[@type="domain"]', namespaces=root.nsmap)
                if usg is not None:
                    paragraph_domain = usg.text
                else:
                    paragraph_domain = None

                # Extract the text content from the paragraph element
                content = extract_text(paragraph)
                html = extract_xml_content(paragraph)

                if i == 0:
                    content = entry_lemma + " " + content
                    html = entry_lemma + " " + html
                #entry_content += content + "\n\n"

                #if entry_content != "":
                row = [volume, entry_id, entry_lemma, id, 'mainEntry', i+1, paragraph_domain, content, html, num_usg_domain]
                #volume | entry | entry_lemma | paragraph | paragraph_domain | content
                data.append(row)
            

            for relatedEntry in entry.findall('.//entry[@type="relatedEntry"]', namespaces=root.nsmap):
                id += 1
            
                form = relatedEntry.find('.//form/orth', namespaces=root.nsmap)
                if form is not None:
                    paragraph_lemma = form.text
                else:
                    paragraph_lemma = None
                
                for i, paragraph in enumerate(relatedEntry.findall('.//p', namespaces=root.nsmap)):
                    usg = None
                    content = ""
                    html = ""

                    form = paragraph.find('.//form/orth', namespaces=root.nsmap)
                    if form is not None:
                        paragraph_lemma = form.text
                    else:
                        paragraph_lemma = None
                    # get number of usg with type="domain" in the paragraph
                    num_usg_domain = len(paragraph.findall('.//usg[@type="domain"]', namespaces=root.nsmap))

                    usg = paragraph.find('.//usg[@type="domain"]', namespaces=root.nsmap)
                    if usg is not None:
                        paragraph_domain = usg.text
                    else:
                        paragraph_domain = None

                    # Extract the text content from the paragraph element
                    content = extract_text(paragraph)
                    html = extract_xml_content(paragraph)

                    if i == 0:
                        content = entry_lemma + " " + content
                        html = entry_lemma + " " + html

                    #relatedEntry_content += content + "\n\n"

                    row = [volume, entry_id, entry_lemma, id, 'relatedEntry', i+1, paragraph_domain, content, html, num_usg_domain]
                    #volume | entry | entry_lemma | subordinate | subordinate_domain | content
                    data.append(row)
                    
                #print(row)
        
    except etree.XMLSyntaxError as e:
        print(f"Erreur de syntaxe XML : {e}")


# convert data into a dataframe

df = pd.DataFrame(data, columns=['volume', 'entry', 'head', 'subEntryId', 'type', 'paragraphId', 'srcDomain', 'text', 'html', 'numUsgDomain'])
df.head()


Erreur de syntaxe XML : Document is empty, line 1, column 1 (.DS_Store, line 1)
TR1.tei
TR2.tei
TR3.tei
TR4.tei
TR5.tei
TR6.tei


,volume,entry,head,subEntryId,type,paragraphId,srcDomain,text,html,numUsgDomain
0,1,250000010,A,1,mainEntry,1,None,A est la première Lettre de l'Alphabet Françoi...,A \n est la première Lettre de l'Alp...,0
1,1,250000010,A,1,mainEntry,2,None,C'est inutilement que la plupart des Grammairi...,\n C'est inutilement que la plupart d...,0
2,1,250000010,A,1,mainEntry,3,None,"A se prononce du gozier, ce qui ne rend pas ce...","\n A se prononce du gozier, ce qui ne...",0
3,1,250000010,A,1,mainEntry,4,None,Le son de l'a est ordinairement un son clair. ...,\n Le son de l'a est ordinairement un...,0
4,1,250000010,A,1,mainEntry,5,None,Le son de l'a est un de ceux que les muets for...,\n Le son de l'a est un de ceux que l...,0


In [19]:
### !!! CODE BELOW IS FOR SUBENTRY LEVEL (NOT PARAGRAPH LEVEL)
inputpath = os.path.join('..', 'data', '1743_LeRobert', 'tei')

data = []

for filename in sorted(os.listdir(inputpath)):
    try:
        parser = etree.XMLParser(collect_ids=False, encoding='utf-8') 
        root = etree.parse(os.path.join(inputpath, filename), parser=parser).getroot()    
        #print(root.nsmap)
        print(filename)
        volume = filename[2:3]

        for entry in root.findall('.//entry[@type="mainEntry"]', namespaces=root.nsmap):
            usg = None
            id = 1
            entry_id = entry.get('id')
            form = entry.find('.//form[@type="lemma"]/orth', namespaces=root.nsmap)
            if form is not None:
                entry_lemma = form.text
            else:
                print("Forme : non trouvée, entry:", entry_id)
                entry_lemma = None

            #if entry_lemma == "AAHUS":
            #        print("Trouvé AAHUS dans entry:", entry_id, "paragraph:", i+1)
            
           
            entry_content = ""
            entry_html = ""
            lemmaGrp = entry.find('.//form[@type="lemmaGrp"]', namespaces=root.nsmap)
            if lemmaGrp is not None:
                entry_content = extract_text(lemmaGrp)
                # manicule: &#9758; <g ref="#manicule-glyph"/>
                manicule = lemmaGrp.find('.//g[@ref="#manicule-glyph"]', namespaces=root.nsmap)
                if manicule is not None:
                    entry_html = "&#9758; " + entry_content + " "
                else:
                    entry_html = entry_content + " "
          
            for i, paragraph in enumerate(entry.findall('./p', namespaces=root.nsmap)):
                
                content = ""
                html = ""
                manicule = paragraph.find('.//g[@ref="#manicule-glyph"]', namespaces=root.nsmap)
                if manicule is not None:
                    html = "&#9758; "
                
                form = paragraph.find('.//form/orth', namespaces=root.nsmap)
                if form is not None:
                    paragraph_lemma = form.text
                else:
                    paragraph_lemma = None

                
                if usg is None:
                    usg = paragraph.find('.//usg[@type="domain"]', namespaces=root.nsmap)
                    if usg is not None:
                        paragraph_domain = usg.text
                    else:
                        paragraph_domain = None

                # Extract the text content from the paragraph element
                content += extract_text(paragraph)
                html += extract_xml_content(paragraph)

                #if i == 0:
                #    content = entry_lemma + " " + content
                #    html = entry_lemma + " " + html

                entry_content += content + "\n\n"
                entry_html += html + "</p><p>"

            if entry_content != "":
                row = [volume, entry_id, entry_lemma, id, 'mainEntry', paragraph_domain, entry_content, entry_html]
                #volume | entry | entry_lemma | paragraph | paragraph_domain | content
                data.append(row)
                

            for relatedEntry in entry.findall('.//entry[@type="relatedEntry"]', namespaces=root.nsmap):
                id += 1
                usg = None
                relatedEntry_content = ""
                relatedEntry_html = ""

                form = relatedEntry.find('.//form/orth', namespaces=root.nsmap)
                if form is not None:
                    paragraph_lemma = form.text
                    relatedEntry_content = paragraph_lemma
                else:
                    paragraph_lemma = None


                for i, paragraph in enumerate(relatedEntry.findall('.//p', namespaces=root.nsmap)):
                    
                    manicule = paragraph.find('.//g[@ref="#manicule-glyph"]', namespaces=root.nsmap)
                    if manicule is not None:
                        relatedEntry_html = "&#9758; " + relatedEntry_content
                    else:
                        relatedEntry_content = extract_text(lemmaGrp)


                    form = paragraph.find('.//form/orth', namespaces=root.nsmap)
                    if form is not None:
                        paragraph_lemma = form.text
                    else:
                        paragraph_lemma = None

                    usg = paragraph.find('.//usg[@type="domain"]', namespaces=root.nsmap)
                    if usg is not None:
                        paragraph_domain = usg.text
                    else:
                        paragraph_domain = None

                    # Extract the text content from the paragraph element
                    content = extract_text(paragraph)
                    html = extract_xml_content(paragraph)

                    #if i == 0:
                    #    content = entry_lemma + " " + content
                    #    html = entry_lemma + " " + html

                    relatedEntry_content += content + "\n\n"
                    relatedEntry_html += html + "</p><p>"

                row = [volume, entry_id, entry_lemma, id, 'relatedEntry', paragraph_domain, relatedEntry_content, relatedEntry_html]
                #volume | entry | entry_lemma | subordinate | subordinate_domain | content
                data.append(row)
                    
                #print(row)
        
    except etree.XMLSyntaxError as e:
        print(f"Erreur de syntaxe XML : {e}")


# convert data into a dataframe

df = pd.DataFrame(data, columns=['volume', 'entry', 'head', 'subEntryId', 'type', 'srcDomain', 'text', 'html'])
df.head()


Erreur de syntaxe XML : Document is empty, line 1, column 1 (.DS_Store, line 1)
TR1.tei
TR2.tei
TR3.tei
TR4.tei
TR5.tei
TR6.tei


,volume,entry,head,subEntryId,type,srcDomain,text,html
0,1,250000010,A,1,mainEntry,None,Aest la première Lettre de l'Alphabet François...,A est la première Lettre de l'Alphabet Françoi...
1,1,250000020,AAHUS,1,mainEntry,None,"AAHUS , s.Aahusium. Ville de l'Evéché de Munst...","AAHUS , s. <i>Aahusium.</i> Ville de l'Evéché ..."
2,1,250000030,AAR,1,mainEntry,None,"AAR ,ou AHR. subst. Aara, Abrinca. Rivière d'A...","AAR , ou AHR. subst. <i>Aara, Abrinca.</i> Riv..."
3,1,250000040,AAR,1,mainEntry,None,"AAR ,Arula ou Arola, & non pas Arosa, comme on...","AAR , Arula ou Arola, & non pas <i>Arosa,</i> ..."
4,1,250000050,AARBRER,1,mainEntry,None,"AARBRER ,Terme ancien qui n'est pas aujourd'hu...","AARBRER , Terme ancien qui n'est pas aujourd'h..."


In [20]:
df[df['html'].str.contains(r'<pb')]

,volume,entry,head,subEntryId,type,srcDomain,text,html
0,1,250000010,A,1,mainEntry,None,Aest la première Lettre de l'Alphabet François...,A est la première Lettre de l'Alphabet Françoi...
3,1,250000040,AAR,1,mainEntry,None,"AAR ,Arula ou Arola, & non pas Arosa, comme on...","AAR , Arula ou Arola, & non pas <i>Arosa,</i> ..."
15,1,250000150,ABADIR,1,mainEntry,terme de Mythologie,"ABADIR ,ou ABADDIR ; car Priscien, qui nous a ...","ABADIR , ou ABADDIR ; car Priscien, qui nous a..."
33,1,250000240,ABANA,1,mainEntry,None,"ABANA , s. f.Abana. 4° Reg. v. 12. Riviere de ...","ABANA , s. f. <i>Abana.</i> 4° <i>Reg.</i> v. ..."
45,1,250000270,ABANDONNER,8,relatedEntry,None,"ABANDONNER , verb. act.On dit aussi Abandonne ...","Abandonné, ée . part. pass. & adject. <i>Derel..."
...,...,...,...,...,...,...,...,...
83285,6,250538400,"ZEST, ZESTE",3,relatedEntry,None,"ZEST, ZESTE . s. m.Zest , est aussi un petit m...","Zest , est aussi un petit morceau de pelure d'..."
83339,6,250538910,"ZOARA, ZOARAT",1,mainEntry,None,"ZOARA, ZOARAT . s. m.Nom propre d'une ville de...","ZOARA, ZOARAT . s. m. Nom propre d'une ville d..."
83357,6,250539090,ZÔNE,1,mainEntry,Terme de Géographie & d'Astronomie,ZÔNE . s. f.Terme de Géographie & d'Astronomie...,ZÔNE . s. f. Terme de Géographie & d'Astronomi...
83374,6,250539260,ZUENZIGA,1,mainEntry,None,ZUENZIGA . s. m.Nom propre de Royaume. Zuenzig...,ZUENZIGA . s. m. Nom propre de Royaume. <i>Zue...


In [13]:
df[df['head'] == "ACCOMPAGNATEUR"]

,volume,entry,head,subEntryId,type,srcDomain,text,html
528,1,250003450,ACCOMPAGNATEUR,1,mainEntry,None,ACCOMPAGNATEUR . s. m.Celui qui dans un concer...,&#9758; ACCOMPAGNATEUR . s. m. Celui qui dans ...


In [143]:
df[df['head'] == "ACROSTICHE"].iloc[0,7]

"ACROSTICHE .Ménage le fait masculin, après S. Amant. Quelques-uns le font féminin : l'Académie Françoise a décidé pour le masculin. Sorte de Poësie disposée de telle façon, que chacun des vers commence par une lettre qui fait partie d'un nom qu'on écrit de travers à la marge, afin que chaque lettre du nom réponde à chaque vers. <i>Acrostichis</i> On en fait aussi où le même nom se trouve au milieu, ou aux autres endroits des vers. On a vu même des Sonnets pentacrostiches, où il y avoit cinq <i>acrostiches.</i> Cette sorte de Poësie est aujourd'hui fort méprisée, & un faiseur d'<i>acrostiches</i> est un Poëte ridicule. C'est l'effort & l'application d'un petit esprit. Ce mot vient du mot Grec , <i>summus,</i> ce qui est à une des extrémités, & , vers. Voici un exemple d'<i>acrostiche</i> tout propre à faire sentir combien ces sortes de pièces gênent l'esprit, parce qu'outre l'<i>acrostiche</i> du nom du Roi au commencement des vers, il y a encore des échos à la fin. Il fut fait après l

In [17]:
df.shape

(81374, 8)

In [130]:
df.shape

(81379, 8)

In [149]:
df.shape

(83413, 8)

In [151]:
len(df[df.volume=='1'])

13337

In [103]:
df.head(70)

,volume,entry,head,subEntryId,type,srcDomain,text,html
0,1,250000010,A,1,mainEntry,None,A est la première Lettre de l'Alphabet Françoi...,A est la première Lettre de l'Alphabet Françoi...
1,1,250000020,AAHUS,1,mainEntry,None,AAHUS Aahusium. Ville de l'Evéché de Munster. ...,AAHUS <i>Aahusium.</i> Ville de l'Evéché de Mu...
2,1,250000030,AAR,1,mainEntry,None,"AAR ou AHR. subst. Aara, Abrinca. Rivière d'Al...","AAR ou AHR. subst. <i>Aara, Abrinca.</i> Riviè..."
3,1,250000040,AAR,1,mainEntry,None,"AAR Arula ou Arola, & non pas Arosa, comme on ...","AAR Arula ou Arola, & non pas <i>Arosa,</i> co..."
4,1,250000050,AARBRER,1,mainEntry,None,AARBRER Terme ancien qui n'est pas aujourd'hui...,AARBRER Terme ancien qui n'est pas aujourd'hui...
...,...,...,...,...,...,...,...,...
65,1,250000510,ABATTEMENT,1,mainEntry,None,"ABATTEMENT Foiblesse, manque de force. Defecti...","ABATTEMENT Foiblesse, manque de force. <i>Defe..."
66,1,250000510,ABATTEMENT,2,relatedEntry,termes de Blason,"ABATTEMENT Abattement , se dit figurément en M...","ABATTEMENT Abattement , se dit figurément en M..."
67,1,250000520,ABATTEUR,1,mainEntry,None,"ABATTEUR Qui abat, qui fait tomber. Eversor. C...","ABATTEUR Qui abat, qui fait tomber. <i>Eversor..."
68,1,250000530,ABATTIS,1,mainEntry,None,"ABATTIS Démolition, renversement, ruine. Evers...","ABATTIS Démolition, renversement, ruine. <i>Ev..."


In [21]:
df['book'] = 'DUFLT_1743'

#df.rename(columns={"entry_lemma": "head", "content": "text", "subordinate_domain":"src-domain"}, inplace=True)
df['numero'] = df.groupby('volume')['entry'].transform(lambda x: pd.factorize(x)[0] + 1)

#df = df[['book', 'volume', 'numero', 'head', 'subEntryId',  'type', 'paragraphId','srcDomain', 'text', 'html', 'numUsgDomain']]
df = df[['book', 'volume', 'numero', 'head', 'subEntryId',  'type', 'srcDomain', 'text', 'html']]


In [22]:
df.head(60)

,book,volume,numero,head,subEntryId,type,srcDomain,text,html
0,DUFLT_1743,1,1,A,1,mainEntry,None,Aest la première Lettre de l'Alphabet François...,A est la première Lettre de l'Alphabet Françoi...
1,DUFLT_1743,1,2,AAHUS,1,mainEntry,None,"AAHUS , s.Aahusium. Ville de l'Evéché de Munst...","AAHUS , s. <i>Aahusium.</i> Ville de l'Evéché ..."
2,DUFLT_1743,1,3,AAR,1,mainEntry,None,"AAR ,ou AHR. subst. Aara, Abrinca. Rivière d'A...","AAR , ou AHR. subst. <i>Aara, Abrinca.</i> Riv..."
3,DUFLT_1743,1,4,AAR,1,mainEntry,None,"AAR ,Arula ou Arola, & non pas Arosa, comme on...","AAR , Arula ou Arola, & non pas <i>Arosa,</i> ..."
4,DUFLT_1743,1,5,AARBRER,1,mainEntry,None,"AARBRER ,Terme ancien qui n'est pas aujourd'hu...","AARBRER , Terme ancien qui n'est pas aujourd'h..."
5,DUFLT_1743,1,6,AARON,1,mainEntry,None,AARON . s. m.Aaron. Nom propre qu'il faut pron...,AARON . s. m. Aaron. Nom propre qu'il faut pro...
6,DUFLT_1743,1,7,AB,1,mainEntry,None,"AB ,Cinquième mois des Hébreux, qui répond à n...","AB , Cinquième mois des Hébreux, qui répond à ..."
7,DUFLT_1743,1,7,AB,2,relatedEntry,None,"AB ,Ab , en Langue Syriaque, le dernier mois d...","Ab , en Langue Syriaque, le dernier mois de l'..."
8,DUFLT_1743,1,8,ABA,1,mainEntry,None,"ABA , s.Aba, ou Abae. C'est le nom d'une ville...","ABA , s. <i>Aba,</i> ou <i>Abae.</i> C'est le ..."
9,DUFLT_1743,1,9,ABA,1,mainEntry,None,"ABA ,ou Anba, Pere, titre que les Églises Syri...","ABA , ou Anba, <i>Pere,</i> titre que les Égli..."


In [23]:
df.to_csv(os.path.join('..', 'data', '1743_LeRobert', 'Trevoux1743_html_260225.tsv'), sep='\t', index=False, encoding='utf-8')
#df.to_excel(os.path.join('..', 'data', '1743_LeRobert', 'Trevoux1743_html_260212.xlsx'), index=False)

In [ ]:
df.to_csv(os.path.join('..', 'data', '1743_LeRobert', 'Trevoux1743_paragraphs_260210.tsv'), sep='\t', index=False, encoding='utf-8')
df.to_excel(os.path.join('..', 'data', '1743_LeRobert', 'Trevoux1743_paragraphs_260210.xlsx'), index=False)

In [17]:
df.shape

(83413, 9)